This contains function related to the logictic regression.  
This contains a function <name> which accepts train set and generates x and b.

In [1]:
import numpy as np
# %load_ext nb_black

In [6]:
def optimize(X):
    rwise_mean = np.nanmean(X, axis = 1, keepdims=True)
    rwise_std = np.nanstd(X, axis=1, keepdims=True)
    rwise_std[rwise_std == 0] = 1
    return (X - rwise_mean)/(rwise_std)

In [3]:
def sigmoid(X):
    return 1/(1+ np.exp(-X))

In [4]:
def initialize_wb(dim):
    w = np.random.randn(dim,1)
    # w = np.zeros((dim,1))
    b = 0
    return w,b

In [5]:
def costFunc(A,Y):
    m = Y.shape[1]
    # assert(not np.any( A*(1-A) ==0)), "A cant have 0 and 1"
    cost = (-1/m)*np.sum(Y*np.log(A) + (1-Y)*np.log(1-A))
    return np.squeeze(cost)

Implementing a function `propagate()` that computes the cost function and its gradient.

**Hints**:

Forward Propagation:
- You get X
- You compute $A = \sigma(w^T X + b) = (a^{(1)}, a^{(2)}, ..., a^{(m-1)}, a^{(m)})$
- You calculate the cost function: $J = -\frac{1}{m}\sum_{i=1}^{m}y^{(i)}\log(a^{(i)})+(1-y^{(i)})\log(1-a^{(i)})$

Here are the two formulas you will be using: 

$$ \frac{\partial J}{\partial w} = \frac{1}{m}X(A-Y)^T\tag{7}$$
$$ \frac{\partial J}{\partial b} = \frac{1}{m} \sum_{i=1}^m (a^{(i)}-y^{(i)})\tag{8}$$

In [7]:
def propagate(X,Y,w,b):
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) +b)
    cost =costFunc(A,Y)
    #calculating  gradients now
    grads = {}
    grads["dw"] = (1/m)*np.dot(X, (A-Y).T)
    grads["db"] = (1/m)*np.sum(A-Y)
    assert(grads["db"] == grads["db"])
    #asserts for debugging
    assert(grads["dw"].shape == w.shape)
    assert(grads["db"].dtype == float)
    assert(cost.shape == ())
    
    return grads,cost

    

In [7]:
def regressionRun(w, b, X, Y, num_iterations, learning_rate, print_cost = True):
    """
    Optimizes w and b by running gradient descent algorithm
    
    Arguments:
    w : intial weights, a numpy array of size (nx, 1)
    b : initial bias, a scalar
    X : data of shape (nx, number of examples)
    Y : boolean label vector
    num_iterations : number of iterations of the optimization loop
    learning_rate : learning rate of the gradient descent update rule
    print_cost : True to print the loss every 100 steps
    
    Returns: params, grads, costs
    > params : dictionary containing the weights w and bias b
    > grads : dictionary containing the gradients of the weights and bias with respect to the cost function
    > costs : list of costs (every 100 iter) computed during the optimization, be used to plot the learning curve.
        """
    costs = []
    grads = {}
    for i in range(num_iterations):
        grads, cost = propagate(X,Y,w,b)
        dw = grads["dw"]
        db = grads["db"]

        w -= learning_rate*dw
        b -= learning_rate*db

        if(i%100 == 0):
            costs.append(cost)
            if(print_cost):
                print("Cost after iteration {0}: {1}".format(i,cost))
    params = {"w": w, "b": b}
    return params, grads, costs        

In [3]:
def predict(X,w,b):
    """
        Predicts Y vector given w,b parameters from data set X
        X: np.array of shape (nx, m)
        w: np.array of shape (1,nx)
        b: real num
        Y: np.array of shape (1,m)
    """
    m = X.shape[1]
    A = sigmoid(np.dot(w.T, X) +b)
    print(A)
    Y_pred = np.zeros((1,m))
    assert(A.shape == Y_pred.shape)
    for i in range(m):
        Y_pred[0][i] = 1 if(A[0][i] > 0.5) else 0
    return Y_pred

In [1]:
def model(X_train, Y_train, X_test, Y_test, num_iterations = 2000, learning_rate = 0.5, print_cost = True):
    """
    Builds the logistic regression model by calling the functions implemented previously
    
    Arguments:
    X_train : training set represented by a numpy array of shape (nx, m_train)
    Y_train : training labels represented by a numpy array (vector) of shape (1, m_train)
    X_test : test set represented by a numpy array of shape (nx, m_test)
    Y_test : test labels represented by a numpy array (vector) of shape (1, m_test)
    num_iterations : hyperparameter representing the number of iterations to optimize the parameters
    learning_rate : hyperparameter representing the learning rate used in the update rule of optimize()
    print_cost : Set to true to print the cost every 100 iterations
    
    Returns: d : dictionary containing information about the model, i.e 
        "costs",
        "Y_prediction_test" 
        "Y_prediction_train" 
        "w" : w, 
        "b" : b,
        "learning_rate" : learning_rate,
        "num_iterations"
    """
    m = X_train.shape[1]
    nx = X_train.shape[0]
    X_train = optimize(X_train)
    X_test = optimize(X_test)
    w,b = initialize_wb(nx)
    params, grads, costs = regressionRun(w,b,X_train, Y_train, num_iterations, learning_rate, print_cost = print_cost)
    w = params["w"]
    b = params["b"]

    Y_prediction_train = predict(X_train, w, b)
    Y_prediction_test = predict(X_test, w, b)

    print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
    print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
    d = {
        "Y_prediction_train" : Y_prediction_train,
        "Y_prediction_test" : Y_prediction_test,
        "w": w,
        "b": b,
        "learning_rate" : learning_rate,
        "num_iterations" : num_iterations
    }
    return d
